In [1]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re
from vicuna_setup import vicuna_inference
from vicuna_pubmedqa_eval import model_pubmed_evaluator
import pandas as pd
import csv
from collections import Counter
from evaluation import pub_scores

/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so.11.0'), PosixPath('/home/minigpt_env/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
path="/home/vicuna-weights-7B"
model_path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)

In [3]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.06s/it]


In [11]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

pubmed_config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=3,
        min_length=1,
        max_time=3,
        max_new_tokens=3, # 3
        forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [10]:
#ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=0)

In [ ]:
len(ans_dict)

In [ ]:
with open("/home/medical-llama/pubmed-pred/base.json", "w") as json_file:
    json.dump(ans_dict, json_file)

In [ ]:
pub_scores("/home/medical-llama/pubmed-pred/base.json")

In [ ]:
ans_dict

In [ ]:
acc,f1_macro=[],[]
for i in range(5):
    ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=i)
    with open("/home/medical-llama/pubmed-pred/base.json", "w") as json_file:
        json.dump(ans_dict, json_file)
    accuracy,f1=pub_scores("/home/medical-llama/pubmed-pred/base.json")
    acc.append(accuracy)
    f1_macro.append(f1)


In [ ]:
print(acc)
print(f1_macro)

In [ ]:
print("Accuracy: ",np.mean(acc))
print("F1 Macro: ",np.mean(f1_macro))

In [5]:
test_data = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")


In [8]:
config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=128,
        min_length=1,
        max_time=10,
        max_new_tokens=128,)

In [9]:
print(vicuna_inference("can i take paracetamol & drink alcohol ?",model=model,tokenizer=tokenizer,config=config))


It is generally not recommended to mix paracetamol with alcohol as it
can cause stomach problems and potentially lead to liver damage. If
you have any concerns or questions about taking paracetamol, it's best
to consult with a healthcare professional. 


In [12]:
def pubmed_template(Question, Context,id=0):

    prompt_template=f"""
    
    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    """
    return prompt_template